# Vesuvius Challenge - Ink Detection: Submission

In [1]:
import wandb
from glob import glob
from os import sep
from os.path import join, abspath
import os

import cv2
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

MODELS_DIR = join(os.pardir, 'models')
KAGGLE_INPUT_DIR = join(abspath(sep), 'kaggle', 'input')
TEST_FRAGMENTS_PATH = join(KAGGLE_INPUT_DIR, 'vesuvius-challenge-ink-detection', 'test')
TEST_FRAGMENTS_PATH = join(os.pardir, 'data', 'raw', 'test')

## Vesuvius WandB

In [2]:
df = pd.read_csv(join(os.pardir, 'data', 'raw', 'wandb', 'wandb_export.csv'))
df.set_index('ID', inplace=True)
df['start_slice'] = df['start_slice'].astype(int)
df['num_slices'] = df['num_slices'].astype(int)

## Utils

In [3]:
def reconstruct_output(tiles, bboxes, fragment_id, fragment_shape, tile_size):
    reconstructed_output = torch.zeros(fragment_shape).to(device=tiles.device)
    count_map = torch.zeros(fragment_shape).to(device=tiles.device)

    for i in range(tiles.shape[0]):
        x0, y0, x1, y1 = bboxes[i]
        reconstructed_output[y0:y1, x0:x1] += tiles[i, :, :]
        count_map[y0:y1, x0:x1] += 1

    reconstructed_output /= count_map
    reconstructed_output = torch.nan_to_num(reconstructed_output, nan=0)

    mask_path = os.path.join(TEST_FRAGMENTS_PATH, fragment_id, 'mask.png')
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    padding = get_padding(mask.shape, tile_size)

    shape = reconstructed_output.shape
    x0, y0, x1, y1 = padding[1][0], padding[0][0], shape[1] - padding[1][1], shape[0] - padding[0][1]
    reconstructed_output = reconstructed_output[y0:y1, x0:x1]

    return reconstructed_output


def get_fragment_shape(fragment_dir, fragment_id, tile_size):
    mask_path = os.path.join(fragment_dir, fragment_id, 'mask.png')
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    padding = get_padding(mask.shape, tile_size)
    mask_pad = np.pad(mask, padding)

    return mask_pad.shape


def get_padding(mask_shape, tile_size, overlap=0.5):
    pad_left = int(overlap * tile_size)
    pad_up = int(overlap * tile_size)
    pad_right = int(overlap * tile_size + tile_size - mask_shape[1] % tile_size)
    pad_down = int(overlap * tile_size + tile_size - mask_shape[0] % tile_size)
    padding = [(pad_up, pad_down), (pad_left, pad_right)]

    return padding


def get_device():
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif torch.backends.mps.is_available():
        device = 'mps'

    return torch.device(device=device)

## Vesuvius Lightning

In [4]:
import cv2
import pytorch_lightning as pl
import segmentation_models_pytorch as smp

class LightningVesuvius(pl.LightningModule):
    def __init__(self, model_params):
        super().__init__()

        self.model = smp.Unet(**model_params)

    def forward(self, inputs):
        x = self.model(inputs)

        return x

## Vesuvius Dataset

In [5]:
# from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import transforms as T

class DatasetVesuvius(Dataset):
    def __init__(self, fragments, tile_size, num_slices, slices_list, start_slice, reverse_slices, selection_thr, augmentation, device, overlap):
        self.fragments = fragments
        self.tile_size = tile_size
        self.num_slices = num_slices
        self.slices_list = slices_list
        self.start_slice = start_slice
        self.reverse_slices = reverse_slices
        self.selection_thr = selection_thr
        self.augmentation = augmentation
        self.device = device

        self.overlap = overlap
        self.set_path = TEST_FRAGMENTS_PATH
        self.slices = self.make_slices()
        self.data, self.items = self.make_data()

        self.transforms = T.RandomApply(
            nn.ModuleList([
                T.RandomRotation(180),
                T.RandomPerspective(),
                T.ElasticTransform(alpha=500.0, sigma=10.0),
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip()
            ]), p=0.5
        )

    def make_slices(self):
        total_slices = 65
        slices = [i for i in range(total_slices)]

        if self.slices_list:
            slices = self.slices_list
        else:
            slices = sorted(slices[self.start_slice:self.start_slice+self.num_slices], reverse=self.reverse_slices)

        return slices

    def make_mask(self, fragment_path):
        mask_path = os.path.join(fragment_path, 'mask.png')
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        shape = (self.num_slices, mask.shape[0], mask.shape[1])
        padding = get_padding(mask.shape, self.tile_size)
        mask_pad = np.pad(mask, padding)

        return mask_pad, shape, padding

    def make_image(self, fragment_path, shape, padding):
        image = np.zeros(shape=shape, dtype=np.uint8)
        slices_files = sorted(glob(os.path.join(fragment_path, 'surface_volume/*.tif')))
        slices_path = [slices_files[i] for i in self.slices]

        print(f'\nMake image from {fragment_path}')
        for i, slice_path in tqdm(enumerate(slices_path), total=len(slices_path)):
            image[i, ...] = cv2.imread(slice_path, cv2.IMREAD_GRAYSCALE)

        padding.insert(0, (0, 0))
        image_pad = np.pad(image, padding)

        return image_pad

    def create_items(self, fragment, mask_pad):
        items = []
        overlap_size = int(self.overlap * self.tile_size)
        x_list = np.arange(0, mask_pad.shape[1] - overlap_size, overlap_size).tolist()
        y_list = np.arange(0, mask_pad.shape[0] - overlap_size, overlap_size).tolist()

        for x in x_list:
            for y in y_list:
                bbox = torch.IntTensor([x, y, x + self.tile_size, y + self.tile_size])
                x0, y0, x1, y1 = bbox
                tile = mask_pad[y0:y1, x0:x1]

                if tile.sum() / (255 * self.tile_size ** 2) >= self.selection_thr:
                    items.append({'fragment': fragment, 'bbox': bbox})

        return items

    def make_data(self):
        data = {}
        items = []

        for fragment in self.fragments:
            fragment_path = os.path.join(self.set_path, str(fragment))
            mask_pad, shape, padding = self.make_mask(fragment_path)
            image_pad = self.make_image(fragment_path, shape, padding)
            items += self.create_items(fragment, mask_pad)

            data[fragment] = {
                'image': torch.from_numpy(image_pad).to(self.device)
            }

        return data, items

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        fragment, bbox = self.items[idx]['fragment'], self.items[idx]['bbox']
        x0, y0, x1, y1 = bbox
        image = self.data[fragment]['image'][:, y0:y1, x0:x1] / 255.0

        return fragment, bbox, image


## Vesuvius Prediction

Calcul weight for each kfold models

In [6]:
df_weight = df.groupby('start_slice')['val/sub_f05_score'].mean()
df_weight /= df_weight.sum()
display(df_weight)
df_weight.sum()

start_slice
0     0.153861
8     0.154643
16    0.172690
24    0.184504
32    0.180550
40    0.153751
Name: val/sub_f05_score, dtype: float64

1.0000000000000002

Get dataset parameters for a given cross validation

In [7]:
def get_dataset_parameters(start_slice, df):
    device = get_device()
    dataset_params = {
        "fragments": ['a', 'b'],
        "tile_size": df['tile_size'][0],
        "num_slices": df['num_slices'][0],
        "start_slice": start_slice,
        "slices_list": None,
        "reverse_slices": df['reverse_slices'][0],
        "selection_thr": 0,
        "augmentation": False,
        "device": device,
        "overlap": 0.5,
    }
    return dataset_params

get_dataset_parameters(16, df)

{'fragments': ['a', 'b'],
 'tile_size': 256,
 'num_slices': 16,
 'start_slice': 16,
 'slices_list': None,
 'reverse_slices': False,
 'selection_thr': 0,
 'augmentation': False,
 'device': device(type='mps'),
 'overlap': 0.5}

In [8]:
def find_checkpoint(id):
    return glob(join(MODELS_DIR, f'*{id}*.ckpt'))[0]

find_checkpoint('u21mnb2m')

'../models/vibrant-sweep-17-u21mnb2m-32-16-3.ckpt'

Get lightning model parameters

In [9]:
def get_lightning_parameters(id, df):
    lightning_params = {
        # "model_name":  'efficientnet-b5',
        "model_params":  {
            "in_channels": df.loc[id, 'num_slices'],
            "encoder_weights": df.loc[id, 'encoder_weights'],
            "classes": 1
        },
        # "learning_rate":  df.loc[id, 'learning_rate'],
        # "bce_weight":  df.loc[id, 'bce_weight'],
        # "dice_threshold":  df.loc[id, 'dice_threshold'],    
    }

    return lightning_params

get_lightning_parameters('u21mnb2m', df)

{'model_params': {'in_channels': 16,
  'encoder_weights': 'imagenet',
  'classes': 1}}

In [10]:
def get_blank_masks(dataset: DatasetVesuvius):
    blank_masks = {fragment_id: {} for fragment_id in dataset.fragments}
    
    for fragment_id in dataset.fragments:
        inklabels_shape = get_fragment_shape(dataset.set_path, fragment_id, dataset.tile_size)
        blank_masks[fragment_id]['inklabels'] = torch.zeros(inklabels_shape).to(device=dataset.device)
        blank_masks[fragment_id]['count_map'] = torch.zeros(inklabels_shape).to(device=dataset.device)
    
    return blank_masks

In [11]:
def add_inklabels(masks, fragment_id, bbox, inklabels):
    x0, y0, x1, y1 = bbox
    masks[fragment_id]['inklabels'][y0:y1, x0:x1] += inklabels
    masks[fragment_id]['count_map'][y0:y1, x0:x1] += 1
    return masks

In [12]:
def make_sub_prediction(dataset: DatasetVesuvius, model, weight):
    masks = get_blank_masks(dataset)
    sigmoid = torch.nn.Sigmoid()
    model.eval()
    
    for fragment_id, bbox, image in dataset:
        image = torch.unsqueeze(image, dim=0)
        sub_inklabels = model(image)
        sub_inklabels = sigmoid(sub_inklabels)
        sub_inklabels = torch.squeeze(sub_inklabels)
        masks = add_inklabels(masks, fragment_id, bbox, sub_inklabels)
    
    inklabels = {}
    for fragment_id in dataset.fragments:
        inklabels[fragment_id] = masks[fragment_id]['inklabels'] / masks[fragment_id]['count_map']
        inklabels[fragment_id] = torch.nan_to_num(inklabels[fragment_id], nan=0)
        inklabels[fragment_id] /= weight
    
    return inklabels

In [13]:
def get_blank_inklabels(fragments, fragment_path):
    blank_inklabels = {}
    
    for fragment_id in fragments:
        blank_inklabels[fragment_id] = cv2.imread(join(fragment_path, fragment_id, 'mask.png'))
    
    return blank_inklabels

masks = get_blank_inklabels(['a', 'b'], TEST_FRAGMENTS_PATH)
masks['a'].shape

(2727, 6330, 3)

In [14]:
model = None
model_temoins = None
inklabels = get_blank_inklabels(['a', 'b'], TEST_FRAGMENTS_PATH)
device = get_device()

for start_slice, weight in df_weight.items():
    dataset_params = get_dataset_parameters(start_slice, df)
    dataset = DatasetVesuvius(**dataset_params)
    
    run_ids = df[df['start_slice'] == start_slice].index
    for run_id in run_ids.to_numpy():
        torch.cuda.empty_cache()
        
        lightning_params = get_lightning_parameters(run_id, df)
        model_ligthning = LightningVesuvius(**lightning_params)
        
        model_path = find_checkpoint(run_id)
        model_ligthning.load_from_checkpoint(
            model_path, 
            map_location='cpu',
            **lightning_params
        )
        model_pytorch = model_ligthning.model
        model_pytorch.to(device=device)
        
        sub_inklabels = make_sub_prediction(dataset, model_pytorch, weight)
        
        for fragment_id in dataset.fragments:
            inklabels[fragment_id] += sub_inklabels[fragment_id]
        break
    break


Make image from ../data/raw/test/a


/var/folders/44/_2tldmgn73jdx8x5snyxsnvw0000gn/T/ipykernel_10377/2100970209.py:42: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  slices = sorted(slices[self.start_slice:self.start_slice+self.num_slices], reverse=self.reverse_slices)


  0%|          | 0/16 [00:00<?, ?it/s]


Make image from ../data/raw/test/b


  0%|          | 0/16 [00:00<?, ?it/s]

KeyError: 'ID'